In [1]:
import re
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
feature_type = '2'

#1 -> just sentence
#2 -> all text
#3 -> all text + other columns

In [3]:
target_type = 3

#1 -> all label
#2 -> pos + neg
#3 -> pos + neg(negative+netural)

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [6]:
train.head()

,text,sentence,snp,phenotype,ASSOCIATION,CONFIDENCE
0,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,nicotine metabolism,neutral,zero
1,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,fetal growth characteristics,positive,weak
2,OBJECTIVE: Maternal smoking during pregnancy i...,RESULTS: Among mothers who did not smoke durin...,rs1051730,fetal growth characteristic,negative,-
3,OBJECTIVE: Maternal smoking during pregnancy i...,Among mothers who continued smoking during pre...,rs1051730,head circumference,negative,-
4,OBJECTIVE: Maternal smoking during pregnancy i...,The T-allele of maternal rs1051730 was associa...,rs1051730,second and third trimester fetal femur length,positive,weak


In [7]:
dic = {"neutral": "negative"}

train.replace({"ASSOCIATION": dic}, inplace=True)
test.replace({"ASSOCIATION": dic}, inplace=True)

In [8]:
train['label'] = train['ASSOCIATION']
test['label']  = test['ASSOCIATION']

In [9]:
train.dropna(subset=['sentence', 'text'], inplace=True)
test.dropna(subset=['sentence', 'text'], inplace=True)

In [10]:
res = pd.read_excel('res.xlsx')

In [11]:
def vectorize_bow(df_X, cv):
    bow_X = cv.transform(df_X)
    bow_df = pd.DataFrame(data=bow_X.toarray(),
                          columns=[cv.get_feature_names_out()])
    return bow_df

def define_and_run_model(model, X_train, X_test, y_train):
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

def show_result(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))
    print(accuracy_score(y_test, y_pred))
    
def save_res(model_name, feature_type, target_type, y_pred_sr, y_test_sr, idx):
    res_pre = precision_score(y_test_sr, y_pred_sr, average='weighted')
    res_rec = recall_score(y_test_sr, y_pred_sr, average='weighted')
    res_f1 = f1_score(y_test_sr, y_pred_sr, average='weighted')
    res.at[idx, 'model-name'] = model_name
    res.at[idx, 'feature-type'] = feature_type
    res.at[idx, 'target-type'] = target_type
    res.at[idx, 'accuracy'] = accuracy_score(y_test_sr, y_pred_sr)
    res.at[idx, 'precision'] = res_pre
    res.at[idx, 'recall'] = res_rec
    res.at[idx, 'f1-score'] = res_f1

In [12]:
possible_labels = train.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

train['label'] = train.label.replace(label_dict)
test['label']  = test.label.replace(label_dict)

In [13]:
df = pd.concat([train, test])

In [14]:
bow_vectorizer = CountVectorizer(lowercase=False, ngram_range=(1, 1), max_df=.80, min_df=1)
bow_vectorizer.fit(df['text'])

CountVectorizer(lowercase=False, max_df=0.8)

In [15]:
train_bow_df  = vectorize_bow(list(train['text']), bow_vectorizer)
test_bow_df   = vectorize_bow(list(test['text']), bow_vectorizer)

In [16]:
X_train = train_bow_df
y_train = train['label']

X_test = test_bow_df
y_test = test['label']

In [17]:
X_train.shape

(934, 7335)

In [18]:
y_train.shape

(934,)

In [19]:
X_test.shape

(365, 7335)

In [20]:
y_test.shape

(365,)

In [21]:
train['ASSOCIATION'].value_counts()

positive    701
negative    233
Name: ASSOCIATION, dtype: int64

In [22]:
test['ASSOCIATION'].value_counts()

negative    195
positive    170
Name: ASSOCIATION, dtype: int64

In [23]:
%%time
smodel = SVC(kernel='rbf')
smodel.fit(X_train, y_train)
pred_values_svm = smodel.predict(X_test)
show_result(pred_values_svm, y_test)
save_res('svm', feature_type, target_type, pred_values_svm, y_test, len(res))

[[  1 194]
 [  2 168]]
              precision    recall  f1-score   support

           0      0.333     0.005     0.010       195
           1      0.464     0.988     0.632       170

    accuracy                          0.463       365
   macro avg      0.399     0.497     0.321       365
weighted avg      0.394     0.463     0.300       365

0.46301369863013697
Wall time: 5.28 s


In [24]:
%%time
lr = LogisticRegression(random_state=0,
                        penalty='l2',
                        solver='newton-cg',
                        C=1.2,
                        max_iter=20)
lr.fit(X_train, y_train)
pred_values_lr = lr.predict(X_test)
show_result(pred_values_lr, y_test)
save_res('LogisticRegression', feature_type, target_type, pred_values_lr, y_test, len(res))

[[ 26 169]
 [ 18 152]]
              precision    recall  f1-score   support

           0      0.591     0.133     0.218       195
           1      0.474     0.894     0.619       170

    accuracy                          0.488       365
   macro avg      0.532     0.514     0.418       365
weighted avg      0.536     0.488     0.405       365

0.4876712328767123
Wall time: 3.25 s


In [25]:
%%time
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
pred_values_rf = model.predict(X_test)
show_result(pred_values_rf, y_test)
save_res('RandomForest', feature_type, target_type, pred_values_rf, y_test, len(res))

[[  5 190]
 [  2 168]]
              precision    recall  f1-score   support

           0      0.714     0.026     0.050       195
           1      0.469     0.988     0.636       170

    accuracy                          0.474       365
   macro avg      0.592     0.507     0.343       365
weighted avg      0.600     0.474     0.323       365

0.473972602739726
Wall time: 3.91 s


In [26]:
%%time
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)
pred_values_knn = model.predict(X_test)
show_result(pred_values_knn, y_test)
save_res('knn', feature_type, target_type, pred_values_knn, y_test, len(res))

[[ 37 158]
 [ 25 145]]
              precision    recall  f1-score   support

           0      0.597     0.190     0.288       195
           1      0.479     0.853     0.613       170

    accuracy                          0.499       365
   macro avg      0.538     0.521     0.451       365
weighted avg      0.542     0.499     0.439       365

0.4986301369863014
Wall time: 416 ms


In [27]:
%%time
model = GaussianNB()
model.fit(X_train, y_train)
pred_values_nb = model.predict(X_test)
show_result(pred_values_nb, y_test)
save_res('GaussianNB', feature_type, target_type, pred_values_nb, y_test, len(res))

[[ 21 174]
 [  7 163]]
              precision    recall  f1-score   support

           0      0.750     0.108     0.188       195
           1      0.484     0.959     0.643       170

    accuracy                          0.504       365
   macro avg      0.617     0.533     0.416       365
weighted avg      0.626     0.504     0.400       365

0.5041095890410959
Wall time: 381 ms


In [28]:
%%time
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
pred_values_dt = model.predict(X_test)
show_result(pred_values_dt, y_test)
save_res('DecisionTree', feature_type, target_type, pred_values_dt, y_test, len(res))

[[ 43 152]
 [ 50 120]]
              precision    recall  f1-score   support

           0      0.462     0.221     0.299       195
           1      0.441     0.706     0.543       170

    accuracy                          0.447       365
   macro avg      0.452     0.463     0.421       365
weighted avg      0.452     0.447     0.412       365

0.4465753424657534
Wall time: 830 ms


In [29]:
%%time
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(X_train, y_train)
pred_values_gb = model.predict(X_test)
show_result(pred_values_gb, y_test)
save_res('GradientBoosting', feature_type, target_type, pred_values_gb, y_test, len(res))

[[ 25 170]
 [ 15 155]]
              precision    recall  f1-score   support

           0      0.625     0.128     0.213       195
           1      0.477     0.912     0.626       170

    accuracy                          0.493       365
   macro avg      0.551     0.520     0.420       365
weighted avg      0.556     0.493     0.405       365

0.4931506849315068
Wall time: 14.6 s


In [30]:
import os
os.remove("res.xlsx") 

res.to_excel('res.xlsx', index=False)